In [ ]:
from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.mixed_system import MixedSystem
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as linalg

import timeit
import numba

# Notebook to explore better implementations

In [ ]:
def get_testsystem(dims_state,dims_output,dims_input,causal=True,dim_state_in=0):
    if dims_state[-1] != 0:
        print('last states usually is 0')
    past_state=dim_state_in
    stages = []
    for i in range(len(dims_input)):
        A = np.random.rand(dims_state[i],past_state)
        B = np.random.rand(dims_state[i],dims_input[i])
        C = np.random.rand(dims_output[i],past_state)
        D = np.random.rand(dims_output[i],dims_input[i])
        past_state=dims_state[i]
        stages.append(Stage(A,B,C,D))
        
    return StrictSystem(causal,stages=stages)

In [ ]:
def show_system(system):
    #function that uses matshow to display the resulting matrix and also shows the divisions
    mat = system.to_matrix()
    plt.matshow(mat)
    x=-0.5
    y=-0.5
    for st in system.stages:
        x+=st.dim_in
        plt.hlines(y,-0.5,x)
        plt.vlines(x,y,mat.shape[0]-0.5)
        y+=st.dim_out
                   
        

def check_dims(system,dim_state_in=0,dim_state_out=0,text_output=True,return_report=False):
    rep = ""
    correct = True
    dim_state = dim_state_in
    for i,st in enumerate(system.stages):
        #check if the state input is correct for A and C
        if st.A_matrix.shape[1] != dim_state:
            correct = False
            rep = rep + "Problem at index "+str(i)+": State dims of A do not match: old:"+str(dim_state)+ \
                  " new: "+str(st.A_matrix.shape[1])+"\n"
        if st.C_matrix.shape[1] != dim_state:
            correct = False
            rep = rep + "Problem at index "+str(i)+": State dims of C do not match: old:"+str(dim_state)+ \
                  " new: "+str(st.C_matrix.shape[1])+"\n"
            
        #check if the state output of A and B match
        dim_state = st.A_matrix.shape[0]
        if st.B_matrix.shape[0] != dim_state:
            correct = False
            rep = rep + "Problem at index "+str(i)+": State dims of A and B do not match: A:"+str(dim_state)+ \
                  "B: "+str(st.B_matrix.shape[0]) + "\n"
            
        #check if the input dims match
        if st.B_matrix.shape[1] != st.D_matrix.shape[1]:
            correct = False
            rep = rep + "Problem at index "+str(i)+": Input dims of B and D do not match: B:"+str(st.B_matrix.shape[1])+ \
                  "D: "+str(st.D_matrix.shape[1]) +"\n"
        
        #check if the output states match
        if st.C_matrix.shape[0] != st.D_matrix.shape[0]:
            correct = False
            rep = rep + "Problem at index "+str(i)+": Output dims of C and D do not match: C:"+str(st.C_matrix.shape[0])+ \
                  "D: "+str(st.D_matrix.shape[0]) +"\n"
    if dim_state != dim_state_out:
        correct = False
        rep = rep + "final state dim does not match"
    if text_output:
        if correct:
            print("Matrix shapes are correct")
        else:
            print("Matrix shapes are not correct")
            print(rep)
    if return_report:
        return correct,rep
    else:
        return correct


In [ ]:
sys = get_testsystem(1*np.ones(20,'i'),1*np.ones(20,'i'),1*np.ones(20,'i'))

In [ ]:
check_dims(sys,dim_state_out=1)

In [ ]:
inp = np.random.rand(sum(sys.dims_in))

x_ref,y_ref = sys.compute(inp.reshape(-1,1))


# Simple straightforward implementation

In [ ]:
def compute(system,u,x_in):
    y = np.zeros(sum(sys.dims_out))
    di= system.dims_in
    do= system.dims_out
    x = x_in
    i_in = 0
    i_out= 0
    for i,s in enumerate(system.stages):
        y[i_out:i_out+do[i]] = s.C_matrix@x+s.D_matrix@u[i_in:i_in+di[i]]
        x =                    s.A_matrix@x+s.B_matrix@u[i_in:i_in+di[i]]
        i_out = i_out+do[i]
        i_in  = i_in+ di[i]
    return y,x

In [ ]:
xin=np.zeros((0))
y,x = compute(sys,inp,xin)
x_in_vec = xin

In [ ]:
np.max(abs(y_ref.T-y))

In [ ]:
mat = sys.to_matrix()
np.max(abs(mat@inp-y))

# Simple implementation with numba

In [ ]:
@numba.jit
def compute_numba(system,u,x_in):
    y = np.zeros(sum(sys.dims_out))
    di= system.dims_in
    do= system.dims_out
    x = x_in
    i_in = 0
    i_out= 0
    for i,s in enumerate(system.stages):
        y[i_out:i_out+do[i]] = s.C_matrix@x+s.D_matrix@u[i_in:i_in+di[i]]
        x =                    s.A_matrix@x+s.B_matrix@u[i_in:i_in+di[i]]
        i_out = i_out+do[i]
        i_in  = i_in+ di[i]
    return y,x

In [ ]:
xin_vec=np.zeros(0)
y,x = compute_numba(sys,inp,xin_vec)
np.max(abs(y_ref.T-y))

# Implementation with numba and numba typed lists

In [ ]:
#so this aparently did not work..
#lets prepare them differently
#For this we use numbas typed lists and place the stage matrices into it
#to be able to assign we also calcualte the indixes of the inputs and outputs 
As = numba.typed.List()
Bs = numba.typed.List()
Cs = numba.typed.List()
Ds = numba.typed.List()
for s in sys.stages:
    As.append(s.A_matrix)
    Bs.append(s.B_matrix)
    Cs.append(s.C_matrix)
    Ds.append(s.D_matrix)

is_in =np.zeros(len(sys.dims_in)+1,dtype=np.int64)
is_out=np.zeros(len(sys.dims_in)+1,dtype=np.int64)
is_in[1] =sys.dims_in[0]
is_out[1]=sys.dims_out[0]
for i in range(2,len(is_in)):
    is_in[i] =is_in[i-1] +sys.dims_in[i-1]
    is_out[i]=is_out[i-1]+sys.dims_out[i-1]

#@numba.jit(nopython=True)
def compute_split(As,Bs,Cs,Ds,is_in,is_out,u,x_in):
    y = np.zeros(is_out[-1])
    x = x_in
    for i in range(is_in.shape[0]-1):
        y[is_out[i]:is_out[i+1]] = Cs[i]@x+Ds[i]@u[is_in[i]:is_in[i+1]]
        x =                        As[i]@x+Bs[i]@u[is_in[i]:is_in[i+1]]
    return y,x

@numba.jit(nopython=True)
def compute_numba_split(As,Bs,Cs,Ds,is_in,is_out,u,x_in):
    y = np.zeros(is_out[-1])
    x = x_in
    for i in range(is_in.shape[0]-1):
        y[is_out[i]:is_out[i+1]] = Cs[i]@x+Ds[i]@u[is_in[i]:is_in[i+1]]
        x =                        As[i]@x+Bs[i]@u[is_in[i]:is_in[i+1]]
    return y,x



In [ ]:
xin_2d=np.zeros((0,1))
y,x = compute_numba_split(As,Bs,Cs,Ds,is_in,is_out,inp.reshape(-1,1),xin_2d)
print(np.max(abs(y_ref.T-y)))
#compute_numba_split(As,Bs,Cs,Ds,is_in,is_out,inp.reshape(-1,1),xin)

# Time them

In [ ]:
#time them
print("matrix_vec:")
print(timeit.timeit(lambda:mat@inp, number=1000))
print("regular:")
print(timeit.timeit(lambda:sys.compute(inp.reshape(-1,1)), number=1000))
print("simplifyied:")
print(timeit.timeit(lambda:compute(sys,inp,x_in_vec), number=1000))
print("simplifyied numba:")
print(timeit.timeit(lambda:compute_numba(sys,inp,xin_2d), number=1000))
print("simplifyied and split numba:")
print(timeit.timeit(lambda:compute_numba_split(As,Bs,Cs,Ds,is_in,is_out,inp.reshape(-1,1),xin_2d), number=1000))


In [ ]:
%timeit sys.compute(inp.reshape(-1,1))

In [ ]:
%timeit compute_numba_split(As,Bs,Cs,Ds,is_in,is_out,inp,xin)

# If the last version is beeing run with numba we get wrong results

here some code to investigate it:

- no loop but only one stage
- no addition 

In [ ]:
@numba.jit(nopython=True)
def compute_numba_fixed_noadd(As,Bs,Cs,Ds,is_in,is_out,u,x_in):
    y = np.zeros(is_out[-1])
    x = x_in
    i = 0
    y[is_out[i]:is_out[i+1]] = Ds[i]@u[is_in[i]:is_in[i+1]]
    #y[is_out[i]:is_out[i+1]] = Cs[i]@x
    x =                        Bs[i]@u[is_in[i]:is_in[i+1]]
    #x =                        As[i]@x
    print(y)
    return y,x

compute_numba_fixed_noadd(As,Bs,Cs,Ds,is_in,is_out,inp,xin)

In [ ]:
@numba.jit(nopython=True)
def compute_numba_printxin(As,Bs,Cs,Ds,is_in,is_out,u,x_in):
    y = np.zeros(is_out[-1])
    x = x_in
    print(x)
    i = 0
    #y[is_out[i]:is_out[i+1]] = Ds[i]@u[is_in[i]:is_in[i+1]]
    y[is_out[i]:is_out[i+1]] = Cs[i]@x
    x =                        Bs[i]@u[is_in[i]:is_in[i+1]]
    #x =                        As[i]@x
    print(y)
    return y,x

compute_numba_printxin(As,Bs,Cs,Ds,is_in,is_out,inp,xin)

# Possible problem with empty multiplications

In [ ]:
G = np.zeros((3,0))

m = np.zeros(0)


@numba.jit(nopython=True)
def test_numba(G,m):
    l = G@m
    return l
test_numba(G,m)

In [ ]:
G@m

In [ ]:
#np.dot?